In [104]:
### permitted modules ###
import numpy as np
import datetime
import hashlib
#########################

### comments about code and algorithms used (optional) ###

##########################################################

In [105]:
###  utility constants, functions and lookups can be instantiated/built here ###

def get_stats(raw):

	
    
	lines = raw.splitlines()

	wins = []

	line = lines[1]

	lines = lines[2:1001]

	dealers:dict = {}


	npStats = np.zeros((1, 14), dtype=np.int32)
	npTips = np.zeros(1, dtype=np.int32)


	bettors = np.zeros(1000, dtype=np.int32)
	bets = np.zeros(1000, dtype=np.int32)
	payout = np.zeros(1000, dtype=np.int32)
	profit = np.zeros(1000, dtype=np.int32)

	#line = "1,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,1970 January 01 15:01:57.000,Homer,0,5,2500,0,2500,500.0"
	
	line_arr = line.split(',')


	#print(arr)

	# Color
	dealer = line_arr[16]
	tip = int(line_arr[17])

	check_dealer_one(dealer, dealers,npStats, npTips)
	#numbers
	get_dealer_stats_np(dealer, line_arr, dealers, npStats)
	
	get_tips_np(dealer, dealers, tip, npTips)

	bettors[0] = eval(line_arr[18])
	bets[0] = eval(line_arr[19])
	payout[0] = eval(line_arr[20])
	profit[0] = eval(line_arr[-2])

	

	# Read lines 2 to 101
	for line_number, line in enumerate(lines, 1):

		line_arr = line.split(',')


		#print(arr)

		# Color
		dealer = line_arr[16]
		tip = int(line_arr[17])

		asa, npStats, npTips = check_dealer(dealer, dealers, npStats, npTips)
		if not asa:
			a = 1
		#numbers
		get_dealer_stats_np(dealer, line_arr, dealers, npStats)
		
		#tips
		get_tips_np(dealer, dealers, tip, npTips)

		bettors[line_number] = eval(line_arr[18])
		bets[line_number] = eval(line_arr[19])
		payout[line_number] = eval(line_arr[20])
		profit[line_number] = eval(line_arr[-2])


	#print(dealers)

	asa = get_number_analytics(npStats, dealers)
	
	asa += "\n\n\n" + get_tips_analytics(npTips, dealers)

	
	asa += "\n\n\n" +get_average_analytics(bettors, bets, payout, profit)

	#print(asa)

	asa += "\n\n##### END #####"

	return asa



	#print(npStats)
	#print(npTips)





def get_average_analytics(bettors, bets, payout, profit):
	asa = "Average Analytics\n"
	asa += "\nAverage Number of Bettors," + str(np.round(np.mean(bettors), decimals=2))
	asa += "\nAverage Bet Size," + str(np.round(np.mean(bets), decimals=2))
	asa += "\nAverage Bet Size per Bettor," + str(np.round(sum(bets)/sum(bettors), decimals=2))
	asa += "\nAverage Payout," + str(np.round(np.mean(payout), decimals=2))
	asa += "\nAverage Payout per Bettor," + str(np.round(sum(payout)/sum(bettors), decimals=2))
	asa += "\nAverage Profit per Spin," + str(np.round(np.mean(profit), decimals=2))
	return asa

def get_number_analytics(npStats, dealers):
	asa ="Number Analytics\n"

	asa += "\n" + total_number_analytics(npStats=npStats)

	sorted_dealers = dict(sorted(dealers.items()))
	for key, value in sorted_dealers.items():
		asa += "\n" + dealer_number_analytics(key, value, npStats)

	return asa


def total_number_analytics(npStats):

	stats = np.sum(npStats, axis=0)
	asa = "Amount,"
	for i in range(len(stats)):
		asa += str(stats[i]) + ","
	asa = asa[:-1]

	pr100 = calculate_percentages(stats, 1000)
	asa += "\nPercentage,"
	for i in range(len(pr100)):
		asa += str(np.round(pr100[i], decimals=1)) + ","

	return asa[:-1]

def dealer_number_analytics(dealer, dealerIndex, npStats):

	asa = "Amount (" + dealer + ")," 
	for i in range(len(npStats[dealerIndex])):
		asa += str(npStats[dealerIndex, i]) + ","
	asa = asa[:-1]

	pr100 = calculate_percentages(npStats[dealerIndex], 1000)
	asa += "\nPercentage (" + dealer + ")," 
	for i in range(len(pr100)):
		asa += str(np.round(pr100[i], decimals=1)) + ","

	return asa[:-1]

def get_tips_analytics(npTips, dealers):
	asa = "Tip Analytics\n"

	asa += "\n" + total_tips_analytics(npTips)

	sorted_dealers = dict(sorted(dealers.items()))
	for key, value in sorted_dealers.items():
		asa += "\n" + dealer_tips_analytics(key, value, npTips)
	
	return asa

def total_tips_analytics(npTips):
	asa = "Tips (Total)," + str(np.sum(npTips))
	return asa

def dealer_tips_analytics(dealer, dealerIndex, npTips):
	asa = "Tips (" + dealer + ")," + str(npTips[dealerIndex])
	return asa

def check_dealer(dealer: str, dealers:dict, npStats:np.ndarray[np.int32], npTips:np.ndarray[np.int32]):
	if dealer not in dealers:
		dealers[dealer] = len(dealers)
		npStatsNew = np.vstack((npStats, np.zeros(14, dtype=np.int32)), dtype=np.int32)
		#print(npStats.size)
		npTipsNew = np.append(npTips, 0)
		return False , npStatsNew, npTipsNew
	else:
		return True , npStats, npTips
	
def check_dealer_one(dealer: str, dealers:dict, npStats:np.ndarray[np.int32], npTips:np.ndarray[np.int32]):
	if dealer not in dealers:
		dealers[dealer] = 0
		return False
	else:
		return True
	
def get_dealer_stats(dealer:str, line_arr:list, dealers:dict, stats:list):
	index = dealers[dealer]
	for i in range(14):
		stats[index][i] += eval(line_arr[i+1].capitalize())

def get_dealer_stats_np(dealer:str, line_arr:list, dealers:dict, npStats:np.ndarray[np.int32]):
	index = dealers[dealer]
	#print(dealers)
	for i in range(14):
		npStats[index, i] += eval(line_arr[i+1].capitalize())

def get_tips(dealer, dealers, tip, tips):
	index = dealers[dealer]
	tips[index] += tip

def get_tips_np(dealer, dealers, tip, npTips):
	index = dealers[dealer]
	npTips[index] += tip

def calculate_percentages(data, size):
    # Calculate the total sum of the dat
    
    # Calculate the percentages using element-wise division and multiplication
    percentages = np.true_divide(data, size) * 100
    
    return percentages
################################################################################

def read_file_content(filename):
    f = open(filename, 'r', encoding = 'UTF-8')
    content = f.read()
    f.close()
    return content

#raw = read_file_content("data_raw_2.txt")

#get_stats(raw)

In [106]:
### algorithm to compute analytics ###
def add_analytics_to_raw_data(raw):
    ### add logic here
    lines = raw.splitlines()
    lines = lines[:-1]
    raw = "\n".join(lines)

    raw += "\n\n"
    processed = raw + get_stats(raw)
    ###
    return processed ### raw data followed by analytics section (should be the same as the data_analytics_memo_{number}.txt)
######################################

In [107]:
####### !!! DO NOT MODIFY !!! #######
def read_file_content(filename):
    f = open(filename, 'r', encoding = 'UTF-8')
    content = f.read()
    f.close()
    return content

def write_file_content(filename, content):
    f = open(filename, 'w', encoding = 'UTF-8')
    f.write(content)
    f.close()
    return None

def do_analyse_data(data_raw, write_file_name):
    data_analytics = add_analytics_to_raw_data(data_raw)
    
    write_file_content(write_file_name, data_analytics)
    return data_analytics

def analyse_data(file_number = '1'):
    base_filename = 'data_'
    
    ### read raw file and analytics memo file content from files with specified number
    data_raw = read_file_content(base_filename + 'raw_' + str(file_number) + '.txt')
    data_analytics_memo = read_file_content(base_filename + 'analytics_memo_' + str(file_number) + '.txt')
    
    ### generate analytics from raw file
    data_analytics = do_analyse_data(data_raw, base_filename + 'analytics_' + str(file_number) + '.txt')
    
    return data_raw, data_analytics, data_analytics_memo

def evaluate_performance(file_number = '1'):
    raw, analytics, analytics_memo = analyse_data(file_number)
    
    # ### print size and type
    # print(raw, type(raw))
    # print(analytics, type(analytics))
    # print(analytics_memo, type(analytics_memo))
    
    analytics_hash = hashlib.md5(analytics.encode()).hexdigest()
    analytics_memo_hash = hashlib.md5(analytics_memo.encode()).hexdigest()
    
    is_equal = analytics_hash == analytics_memo_hash ## md5 checksum
    
    return is_equal
####### !!! DO NOT MODIFY !!! #######

### PERFORMANCE
REQUIREMENT: Generated output file should be identical to the memo file.

In [108]:
### replace file_number with the number of the txt file you want to test on
file_number = '1'

### run cell to get your score
is_requirement_satisfied = evaluate_performance(file_number)

print('Requirement satisfied: ' + str(is_requirement_satisfied))

Requirement satisfied: True
